<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/4.4.%E6%A8%A1%E5%9E%8B%E8%AF%BB%E5%8F%96%E5%92%8C%E5%AD%98%E5%82%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##4.4. 模型读取和存储
&emsp;&emsp;到目前为止，我们介绍了如何处理数据以及如何构建、训练和测试深度学习模型。然而在实际中，我们有时需要把训练好的模型部署到很多不同的设备。在这种情况下，我们可以把内存中训练好的模型参数存储在硬盘上供后续读取使用。

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential

In [0]:
tf.enable_eager_execution()

### 4.4.1. 模型的存储
&emsp;&emsp;我们先来定义一个模型

In [3]:
x=tf.random.normal(shape=(100,100))
y=tf.random.normal(shape=(100,5))
model=Sequential([layers.Dense(10,input_dim=100,activation='relu'),layers.Dense(5)])
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])
model.fit(x=x,y=y,batch_size=10,epochs=5)

Train on 100 samples
Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
100/100 [==============================] - 3s 33ms/sample - loss: 4.5191 - acc: 0.2200
Epoch 2/5
100/100 [==============================] - 0s 250us/sample - loss: 2.2676 - acc: 0.2200
Epoch 3/5
100/100 [==============================] - 0s 251us/sample - loss: 1.0065 - acc: 0.2100
Epoch 4/5
100/100 [==============================] - 0s 298us/sample - loss: 0.9531 - acc: 0.2100
Epoch 5/5
100/100 [==============================] - 0s 466us/sample - loss: 1.0109 - acc: 0.2200


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                1010      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
Total params: 1,065
Trainable params: 1,065
Non-trainable params: 0
_________________________________________________________________


你可以使用model.save(filepath)将Keras模型和权重保存在一个HDF5文件中，该文件将包含：
* 模型的结构，以便重构该模型
* 模型的权重
* 训练配置（损失函数，优化器等）
* 优化器的状态，以便于从上次训练中断的地方开始

In [0]:
model.save(filepath='my_model.h5')

In [6]:
!ls ./

my_model.h5  sample_data


&emsp;&emsp;如果你只是希望保存模型的结构，而不包含其权重或配置信息，可以使用：

In [0]:
json_string=model.to_json()

In [0]:
yaml_string=model.to_yaml()

In [9]:
json_string

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 100], "dtype": "float32", "units": 10, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 5, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": nu

&emsp;&emsp;这项操作将把模型序列化为json或yaml文件，这些文件对人而言也是友好的，如果需要的话你甚至可以手动打开这些文件并进行编辑。

&emsp;&emsp;如果需要保存模型的权重，可通过下面的代码利用HDF5进行保存。注意，在使用前需要确保你已安装了HDF5和其Python库h5py

In [0]:
model.save_weights('my_model_weights.h5')

In [11]:
!ls ./

my_model.h5  my_model_weights.h5  sample_data


### 4.4.2. 模型的读取
&emsp;&emsp;如果之前使用了model.save()保存了完整的模型结构和权重参数，那么可以使用keras.models.load_model(filepath)来重新实例化你的模型，如果文件中存储了训练配置的话，该函数还会同时完成模型的编译


In [0]:
from tensorflow.keras.models import load_model
model=load_model('my_model.h5')

&emsp;&emsp;你也可以根据之前保存的json或者yaml文件来载入模型结构

In [0]:
from tensorflow.keras.models import model_from_json,model_from_yaml
model=model_from_json(json_string)
model_yaml=model_from_yaml(yaml_string)

In [14]:
model.layers

&emsp;&emsp;然后再加载之前保存的模型权重

In [0]:
model_yaml.load_weights('my_model_weights.h5')

&emsp;&emsp;打印模型结构看看

In [16]:
model_yaml.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                1010      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
Total params: 1,065
Trainable params: 1,065
Non-trainable params: 0
_________________________________________________________________


&emsp;&emsp;如果你需要加载权重到不同的网络结构（有些层一样）中，例如fine-tune或transfer-learning，你可以通过层名字来加载模型：

In [0]:
model=Sequential()
model.add(layers.Dense(10,input_dim=100,name='dense'))
model.add(layers.Dense(3,name='new_dense'))
model.load_weights(filepath='my_model_weights.h5',by_name=True)

In [21]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                1010      
_________________________________________________________________
new_dense (Dense)            (None, 3)                 33        
Total params: 1,043
Trainable params: 1,043
Non-trainable params: 0
_________________________________________________________________


###4.4.3. 小结
* model.save()可以保存完整的模型：包括模型的结构和参数以及优化器等；model.to_json()和model.to_yaml()可以保存模型的网络结构。model.save_weights()可以保存模型的参数。

* load_model()方法可以加载save()方法保存的完整模型；model_from_json(),model_from_yaml()方法可以恢复模型的网络结构，然后可以根据load_weights()方法加载模型。